First run:
~/Work/pipelines/aux.scripts/coverage.sh -resolution -experiment/outdata/q10_chr-loc-strand-umi-pcr -hg19 -sliding


In [5]:
import os
import numpy as np
from scipy import *
from scipy.sparse import *
from scipy.spatial import distance
from scipy.stats import entropy
import pandas as pd
from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute
import matplotlib.pyplot as plt
import seaborn as sns

def sparseload(file,resolution):
    f = open(file)
    h = f.readline()
    col = []
    data = []
    chrom = []
    for l in f:
        l = l.strip()
        chrom.append(l.split('\t')[0])
        col.append(int(l.split('\t')[2])/resolution)
        data.append(int(float(l.split('\t')[3])))
    col = np.array(col)
    data = np.array(data)
    chrom = np.array(chrom)
    f.close()
    return chrom,col,data

def structure_genome(resolution,f):
    base = 0
    positionlist = []
    chromlist = []
    border = []
    for l in f.readlines():
        l = l.strip()
        chrom = l.split('\t')[0]
        size = l.split('\t')[1]
        end = base + int(size)/resolution
        border.append(end)
        array = arange(base, end)
        positionlist.extend(array)
        chromlist.extend([chrom]*len(array))
        base = base + int(size)/resolution
    f.close()
    return positionlist,chromlist,border

def load_data(experiment,positionlist,resolution,sliding):
#     filename = '/home/garner1/Work/dataset/'+str(exp)+'/outdata/q10_chr-loc-strand-umi-pcr-coverage-'+str(resolution)+'_'+str(sliding)+'.bed'
    filename = '/home/garner1/Work/dataset/'+str(exp)+'/outdata/q10_chr-loc-strand-umi-pcr-coverage-'+str(resolution)+'_'+'.bed'
    chrom, col, data = sparseload(filename,resolution)
    return data

resolution = 1*1000000
sliding = 500*1000
RC_threshold = 0
f = open('/home/garner1/Work/pipelines/data/hg19-chr-sizes.txt')
# f = open('/home/garner1/Work/pipelines/data/mm9-chr-sizes.txt')

positionlist,chromlist,border = structure_genome(resolution,f)
exp = 'rm53NT'
X1 = load_data(exp,positionlist,resolution,sliding)
exp = 'rm53T'
X2 = load_data(exp,positionlist,resolution,sliding)
X1_df = pd.Series(X1)
X2_df = pd.Series(X2)
df = pd.concat([X1_df,X2_df],axis=1)
df = df[np.abs(df-df.mean())<=(10*df.std())]
df = df / df.sum(axis=0) 
data = np.asarray(df)

In [6]:
import linecache
filename = '/home/garner1/Work/dataset/rm53T/outdata/q10_chr-loc-strand-umi-pcr-coverage-'+str(resolution)+'_'+str(sliding)+'.bed'
with open("enrichedRM53T_wUMI.bed", "a") as myfile:
    foldChange = data[:,1]/data[:,0]
    for line in np.where(np.logical_and(foldChange>=5, foldChange<=1000))[0]:
        myfile.write(''+ linecache.getline(filename,line) + '')
with open("enrichedRM53NT_wUMI.bed", "a") as myfile:
    foldChange = data[:,0]/data[:,1]
    for line in np.where(np.logical_and(foldChange>=5, foldChange<=1000))[0]:
        myfile.write(''+ linecache.getline(filename,line) + '')
with open("non_enrichedRM53T_wUMI.bed", "a") as myfile:
    foldChange = data[:,1]/data[:,0]
    for line in np.where(np.logical_and(foldChange>=0, foldChange<5))[0]:
        myfile.write(''+ linecache.getline(filename,line) + '')
with open("non_enrichedRM53NT_wUMI.bed", "a") as myfile:
    foldChange = data[:,0]/data[:,1]
    for line in np.where(np.logical_and(foldChange>=0, foldChange<5))[0]:
        myfile.write(''+ linecache.getline(filename,line) + '')


In [27]:
# plt.figure(1)
# plt.plot(data[:,1]/data[:,0],'o')
# # axes = plt.gca()
# # axes.set_ylim([-10,1])
# plt.title('RM53 T vs NT fold changes at '+str(resolution)+'_slide'+str(sliding))
# # plt.savefig('RM53_ToverNT_FC_res'+str(resolution)+'_slide'+str(sliding)+'.png')

# plt.figure(2)
# plt.plot(data[:,0]/data[:,1],'o')
# # axes = plt.gca()
# # axes.set_ylim([-10,1])
# plt.title('RM53 NT vs T fold changes at '+str(resolution)+'_slide'+str(sliding))
# # plt.savefig('RM53_NToverT_FC_res'+str(resolution)+'_slide'+str(sliding)+'.png')

fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.plot(data[:,0]/data[:,1], c='r', label='NT/T')
ax1.plot(data[:,1]/data[:,0], c='b', label='T/NT')
plt.legend(loc='upper left');

plt.show()

In [244]:
plt.figure(1)

g1 = sns.heatmap(data, square=False, annot=False, cmap="Blues", cbar=True)
plt.title('RM53T vs RM53NT DSB probability heatmap at '+str(resolution))
g1.set(yticklabels=[])

plt.show()